In [13]:
from bs4 import BeautifulSoup
import geopandas as gpd
import requests
import pandas as pd

## Open GIS Data

In [14]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)
states_df = states_df.to_crs(4326)

## Retrieve data

In [69]:
url = "https://www.iihs.org/topics/fatality-statistics/detail/state-by-state"

In [70]:
r = requests.get(url)

In [71]:
soup = BeautifulSoup(r.text, "html.parser")

In [72]:
table = soup.find("table")
column_names = [th.text for th in table.find_all("tr")[1].find_all("th")]
trs = table.find_all("tr")[2:]

In [73]:
formatted_records = []
for tr in trs:
    tds = tr.find_all("td")
    state = tr.find("th").text
    formatted_records.append([state, *[float(td.text.replace(",", "")) for td in tds]])

In [79]:
vehicle_df = pd.DataFrame(formatted_records, columns=column_names)
vehicle_df["Deaths"] = vehicle_df["Deaths"].astype(int)
vehicle_df = vehicle_df.rename(columns={"State": "NAME"})

## Merge data

In [85]:
vehicle_gdf = states_df[["NAME", "geometry"]].merge(vehicle_df, on="NAME", how="inner")

## Export

In [86]:
vehicle_gdf = vehicle_gdf.to_crs(9311)

In [87]:
vehicle_gdf.to_file("data/vehicle.gpkg")